In [0]:
 %run ./00_common_imports

In [0]:
%run ./01_autoloader_config

In [0]:
# Databricks notebook source
# MAGIC %run ./00_common_imports
# MAGIC %run ./01_autoloader_config

# COMMAND ----------

from pyspark.sql.functions import current_timestamp

def custom_ingest(source_path, target_path, load_type="full"):
    """
    Custom ingestion logic using PySpark (manual CSV/JSON reader)
    load_type = 'full' or 'incremental'
    """
    try:
        log_info(f"🚀 Starting {load_type.upper()} load from {source_path} to {target_path}")
        
        # 1️⃣ List all files in source
        files = [f.path for f in dbutils.fs.ls(source_path) if f.path.lower().endswith(".csv")]
        if not files:
            log_warning(f"No valid CSV files found in {source_path}")
            return
        
        for file in files:
            log_info(f"Reading file: {file}")
            df = (spark.read
                  .option("header", True)
                  .option("inferSchema", True)
                  .csv(file)
                  .withColumn("ingestion_ts", current_timestamp()))
            
            # 2️⃣ Write to target
            if load_type.lower() == "full":
                df.write.format("delta").mode("overwrite").save(target_path)
                log_info(f"✅ Full load completed successfully for {file}")
            else:
                df.write.format("delta").mode("append").save(target_path)
                log_info(f"✅ Incremental load completed successfully for {file}")

    except Exception as e:
        log_error(f"❌ Error during {load_type.upper()} load: {str(e)}")
        raise

# COMMAND ----------

# Run Full Load
custom_ingest(SOURCE_BASE, VOLUME_BASE, load_type="full")

# COMMAND ----------

# Run Incremental Load
custom_ingest(SOURCE_BASE, VOLUME_BASE, load_type="incremental")
